# Basic usage of LRP
This example will show you best practices for using LRP,
building on the basics shown in the *Getting started* section.

We start out by loading a small convolutional neural network:

In [1]:
using ExplainableAI
using Flux

model = Chain(
    Chain(
        Conv((3, 3), 3 => 8, relu; pad=1),
        Conv((3, 3), 8 => 8, relu; pad=1),
        MaxPool((2, 2)),
        Conv((3, 3), 8 => 16; pad=1),
        BatchNorm(16, relu),
        Conv((3, 3), 16 => 8, relu; pad=1),
        BatchNorm(8, relu),
    ),
    Chain(
        Flux.flatten,
        Dense(2048 => 512, relu),
        Dropout(0.5),
        Dense(512 => 100, softmax)
    ),
);

This model contains two chains: the convolutional layers and the fully connected layers.

## Model preparation
### Stripping the output softmax
When using LRP, it is recommended to explain output logits instead of probabilities.
This can be done by stripping the output softmax activation from the model
using the `strip_softmax` function:

In [2]:
model = strip_softmax(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

If you don't remove the output softmax,
model checks will fail.

### Canonizing the model
LRP is not invariant to a model's implementation.
Applying the `GammaRule` to two linear layers in a row will yield different results
than first fusing the two layers into one linear layer and then applying the rule.
This fusing is called "canonization" and can be done using the `canonize` function:

In [3]:
model = canonize(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

### Flattening the model
ExplainableAI.jl's LRP implementation supports nested Flux Chains and Parallel layers.
However, it is recommended to flatten the model before analyzing it.

LRP is implemented by first running a forward pass through the model,
keeping track of the intermediate activations, followed by a backward pass
that computes the relevances.

To keep the LRP implementation simple and maintainable,
ExplainableAI.jl does not pre-compute "nested" activations.
Instead, for every internal chain, a new forward pass is run to compute activations.

By "flattening" a model, this overhead can be avoided.
For this purpose, ExplainableAI.jl provides the function `flatten_model`:

In [4]:
model_flat = flatten_model(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

This function is called by default when creating an LRP analyzer.
Note that we pass the unflattened model to the analyzer, but `analyzer.model` is flattened:

In [5]:
analyzer = LRP(model)
analyzer.model

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

If this flattening is not desired, it can be disabled
by passing the keyword argument `flatten=false` to the `LRP` constructor.

## LRP rules
By default, the `LRP` constructor will assign the `ZeroRule` to all layers.

In [6]:
LRP(model)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => ZeroRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZeroRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, relu, pad=1) => ZeroRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZeroRule(),
  BatchNorm(8, relu)                 => ZeroRule(),
  Flux.flatten                       => ZeroRule(),
  Dense(2048 => 512, relu)           => ZeroRule(),
  Dropout(0.5)                       => ZeroRule(),
  Dense(512 => 100)                  => ZeroRule(),
)

This analyzer will return heatmaps that look identical to `InputTimesGradient`.

LRP's strength lies in assigning different rules to different layers,
based on their functionality in the neural network[^1].
ExplainableAI.jl implements many LRP rules out of the box,
but it is also possible to *implement custom rules*.

To assign different rules to different layers,
use one of the composites presets,
or create your own composite, as described in
*Assigning rules to layers*.

In [7]:
composite = EpsilonPlusFlat() # using composite preset EpsilonPlusFlat

Composite(
  GlobalTypeMap(  # all layers
    Flux.Conv               => ZPlusRule(),
    Flux.ConvTranspose      => ZPlusRule(),
    Flux.CrossCor           => ZPlusRule(),
    Flux.Dense              => EpsilonRule{Float32}(1.0f-6),
    typeof(NNlib.dropout)   => PassRule(),
    Flux.AlphaDropout       => PassRule(),
    Flux.Dropout            => PassRule(),
    Flux.BatchNorm          => PassRule(),
    typeof(Flux.flatten)    => PassRule(),
    typeof(MLUtils.flatten) => PassRule(),
    typeof(identity)        => PassRule(),
 ),
  FirstLayerTypeMap(  # first layer
    Flux.Conv          => FlatRule(),
    Flux.ConvTranspose => FlatRule(),
    Flux.CrossCor      => FlatRule(),
    Flux.Dense         => FlatRule(),
 ),
)

In [8]:
LRP(model, composite)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => FlatRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZPlusRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, relu, pad=1) => ZPlusRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZPlusRule(),
  BatchNorm(8, relu)                 => PassRule(),
  Flux.flatten                       => PassRule(),
  Dense(2048 => 512, relu)           => EpsilonRule{Float32}(1.0f-6),
  Dropout(0.5)                       => PassRule(),
  Dense(512 => 100)                  => EpsilonRule{Float32}(1.0f-6),
)

## Computing layerwise relevances
If you are interested in computing layerwise relevances,
call `analyze` with an LRP analyzer and the keyword argument
`layerwise_relevances=true`.

The layerwise relevances can be accessed in the `extras` field
of the returned `Explanation`:

In [9]:
input = rand(Float32, 32, 32, 3, 1) # dummy input for our convolutional neural network

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

11-element Vector{Array{Float32}}:
 [0.006043212 0.0066786986 … 0.008904876 0.00060426624; 0.0017615801 0.010971662 … 0.009488763 -0.0058877408; … ; -0.048445586 0.017650932 … -0.00860164 0.0001509917; -0.05323321 -0.063456856 … 0.005115529 0.008618826;;; -0.011974949 0.004015885 … -0.002988963 -0.00015950545; 0.0126312785 -0.008105288 … 0.01044735 0.0032163232; … ; -0.07340901 -0.017870115 … -0.0022665148 -0.011408316; -0.0014210079 -0.01784113 … -0.0014439516 5.9477654f-5;;; -0.0007174437 0.006912653 … -0.0008486262 0.0038422763; 0.007050686 -0.0063280794 … 0.0051072035 0.0011892179; … ; -0.005491156 0.025332095 … -0.0029025495 -0.0028174073; -0.028502006 -0.043011356 … -0.005137534 0.00033960608;;;;]
 [-0.0 -0.0048087304 … -0.00064400793 -0.0014622794; -0.0 0.01375086 … 0.003270082 0.0088651255; … ; 0.002824492 -0.00579066 … -0.009221186 -0.00024642335; -0.020834219 0.010284399 … 0.0066425195 -0.0011724081;;; -0.0 0.0 … 0.0 -0.0; 0.0 0.001332658 … 0.0 -0.0011003235; … ; -0.0 0.0 … -

Note that the layerwise relevances are only kept for layers in the outermost `Chain` of the model.
When using our unflattened model, we only obtain three layerwise relevances,
one for each chain in the model and the output relevance:

In [10]:
analyzer = LRP(model; flatten=false) # use unflattened model

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

11-element Vector{Array{Float32}}:
 [-1.7707294f-5 0.007025882 … 0.0070779864 0.00030808453; 0.0017003585 -0.021291513 … 0.000187628 -0.008914753; … ; -0.019663384 0.013414878 … -0.0068603894 -0.0004869336; -0.04960091 -0.06865828 … -0.011706453 -0.002640587;;; -0.01730596 -0.01869555 … -0.002867833 -0.0010717375; 0.011759536 0.0009975367 … 0.012839309 0.00016412073; … ; -0.06153658 0.008800103 … 0.00084563554 0.025412763; -0.00052466034 -0.016463645 … 0.0058153174 -0.00052558776;;; -0.0025442922 0.014029129 … -0.0015391738 0.0026854558; 0.0009136914 -0.04741096 … 0.007984934 -0.00040405424; … ; 0.008839571 0.018539572 … 0.003629937 0.0006882005; -0.0071480325 -0.018471012 … -0.0056977123 0.0003250995;;;;]
 [0.0 -0.008076934 … -0.0007627126 -0.00029055754; 0.0 0.007848128 … 0.0016436559 0.0056827837; … ; 0.003499025 0.0004156935 … 0.0040776585 -0.0002548854; -0.014737083 0.015108364 … -0.01721048 0.0011025469;;; -0.0 0.0 … 0.0 0.0; 0.0 0.0016062674 … 0.0 -0.0011566371; … ; -0.0 -0.0 … 

## Performance tips
### Using LRP without a GPU
Since ExplainableAI.jl's LRP implementation makes use of
[Tullio.jl](https://github.com/mcabbott/Tullio.jl),
analysis can be accelerated by loading either
- a package from the [JuliaGPU](https://juliagpu.org) ecosystem,
  e.g. [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl), if a GPU is available
- [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
  if only a CPU is available.

This only requires loading the LoopVectorization.jl package before ExplainableAI.jl:
```julia
using LoopVectorization
using ExplainableAI
```

[^1]: G. Montavon et al., [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007/978-3-030-28954-6_10)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*